In [2]:
import sys
sys.path.append("../Modules/")
sys.path.append("../")
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib notebook

    
import logging

#fmt = 'logging.Formatter(''%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
fmt = '%(levelname)s_%(name)s-%(funcName)s(): - %(message)s'
logging.basicConfig(level=logging.INFO, format=fmt)
logger = logging.getLogger(__name__)

np.set_printoptions(precision=2)

In [3]:
import pickle

file = open('../sessionFiles.pickle', 'rb')
allSessions = pickle.load(file)
file.close()

sessionDict = allSessions[0]
sessionDict.keys()

dict_keys(['subID', 'trialInfo', 'expConfig', 'rawExpUnity', 'rawExpGaze', 'processedExp', 'rawCalibUnity', 'rawCalibGaze', 'processedCalib', 'analysisParameters', 'fixAssessmentData'])

In [4]:
os.getcwd()

'D:\\Github\\Interception_UXF_Analysis\\ValidationNotebooks'

In [5]:
 # Get folder/filenames
dataFolderList = []
[dataFolderList.append(name) for name in os.listdir("Data/") if name[0] != '.']

dataFolderList

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Data/'

In [ ]:

    # Get folder/filenames
    dataFolderList = []
    [dataFolderList.append(name) for name in os.listdir("Data/") if name[0] != '.']

    for i, name in enumerate(dataFolderList):
            if i == subNum:
                print('***> ' + str(i) + ': ' + name )
            else:
                print(str(i) + ': ' + name )
    

    dataParentFolder = "Data/" + dataFolderList[subNum]
    dataSubFolderList = []
    [dataSubFolderList.append(name) for name in os.listdir(dataParentFolder) if name[0] != '.']
    dataFolder = dataParentFolder + '/' + dataSubFolderList[0] + '/'

    logger.info('Processing session: ' + dataFolder)

    # Try to load pickle if doNotLoad == False
    picklePath = dataFolder + dataSubFolderList[0] + '.pickle'
    from os import path
    if( doNotLoad == False and path.exists(picklePath)):
        
        file = open(picklePath, 'rb')
        sessionData = pickle.load(file)
        file.close()
        
        logger.info('Importing session dict from pickle.')
        
        return sessionData

    logger.info('Compiling session dict from *.csv.')

    # If not loading from pickle, create and populate dataframes
    rawExpUnityDataDf = pd.DataFrame()
    rawExpGazeDataDf = pd.DataFrame()
    processedExpDataDf = pd.DataFrame()

    rawCalibUnityDataDf = pd.DataFrame()
    rawCalibGazeDataDf = pd.DataFrame()
    processedCalibDataDf = pd.DataFrame()

    trialData = pd.read_csv( dataFolder + '/trial_results.csv')

    for trIdx, trialResults in trialData.iterrows():

        trialDict = processTrial(dataFolder, trialResults,len(trialData))

        def addToDF(targetDF,dfIn):

            return pd.concat([targetDF, dfIn])

            # if( targetDF.empty ):
            #     return dfIn
            # else:
            #     # targetDF = targetDF.append(dfIn)
            #     return pd.concat([targetDF, dfIn])

        if (trialResults['trialType'] == 'interception'):

            processedExpDataDf = addToDF(processedExpDataDf,trialDict['interpolatedData'])
            rawExpUnityDataDf = addToDF(rawExpUnityDataDf,trialDict['rawUnityData'])
            rawExpGazeDataDf = addToDF(rawExpGazeDataDf,trialDict['rawGazeData'])

        elif(trialResults['trialType'] == 'CalibrationAssessment'):

            processedCalibDataDf = addToDF(processedCalibDataDf,trialDict['interpolatedData'])
            rawCalibUnityDataDf = addToDF(rawCalibUnityDataDf,trialDict['rawUnityData'])
            rawCalibGazeDataDf = addToDF(rawCalibGazeDataDf,trialDict['rawGazeData'])

    # Rename trialdata columns
    trialData.rename(columns={"session_num":"sessionNumber","trial_num":"trialNumber",
        "block_num":"blockNumber","trial_num_in_block":"trialNumberInBlock",
        "start_time":"startTime","end_time":"endTime"},inplace=True)

    trDataFiles = [i for i in trialData.columns.to_list() if '_filename' in i] 

    # Convert to multiindex
    newColList = [convertIndexToMultiIndexUsingUnderscore(c) for c in trialData.columns[:-len(trDataFiles)]]
    newColList.extend([convertIndexToMultiIndexIgnoringUnderscore(c) for c in trialData.columns[-len(trDataFiles):]])
    trialData.columns = pd.MultiIndex.from_tuples(newColList)
    
    expDict = json.load( open(dataFolder + '/settings/' + 'settings.json'))

    processedExpDataDf = processedExpDataDf.reset_index(drop=True)
    processedCalibDataDf = processedCalibDataDf.reset_index(drop=True)
    
    analysisParameters = json.load( open('analysisParameters.json'))
    # analysisParameters['gazeDataConfidenceThreshold'] = gazeDataConfidenceThreshold

    
    #logger.warning('(**********************  SUBID FILE IS HARDCODED *******************************')
    #subID = json.load( open(dataParentFolder + '/participantdetails/participant_details.csv'))['ppid']
    subID = trialData['ppid'][1] 

    dictOut = {"subID": subID, "trialInfo": trialData.sort_index(axis=1),"expConfig": expDict,
        "rawExpUnity": rawExpUnityDataDf.sort_index(axis=1), "rawExpGaze": rawExpGazeDataDf.sort_index(axis=1), "processedExp": processedExpDataDf.sort_index(axis=1),
        "rawCalibUnity": rawCalibUnityDataDf.sort_index(axis=1), "rawCalibGaze": rawCalibGazeDataDf.sort_index(axis=1), "processedCalib": processedCalibDataDf.sort_index(axis=1),
        "analysisParameters":analysisParameters}

    with open(picklePath, 'wb') as handle:
        pickle.dump(dictOut, handle, protocol=pickle.HIGHEST_PROTOCOL)